In [1]:
from translator import translate
from transformers import pipeline
import pandas as pd
import numpy as np
from src.functions import check,tresh,license_plate
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc


2022-05-03 14:40:47.718018: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 14:40:47.718036: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

df=pd.read_excel('personal_data_db.xlsx')
df=df.dropna()
df['label']=df['label'].astype(int)
df

,sentence,label
0,"Je m’appelle Laval Jacquin, j’ai 38 ans, mon n...",1
1,Nour Kired et Damien Sonneville sont de bons a...,1
2,Arnaud Mimart est le chef de la chocoPolice,1
3,yeah that's what i i just did today i got uh D...,0
4,"Fondé en 1861, le Massachusetts Institute of T...",0
...,...,...
271,You'll find these in various sizes and with di...,0
272,"[ In all fairness, it must be said that Mr. Ro...",0
273,"eh bien j'arrive ici ce matin et um, j'oublie ...",0
274,c'était depuis Wills Point. Je ne sais pas si ...,0


In [3]:
# candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
name=["First and last name","full name", "person's name","first name and Family name","person"]
birth=["birthday","private"]
# ip=["Internet protocol (IP) addresses","IP adress"]
cookie=["cookie identifiers","a cookie"]
phone=["phone numbers","phone number","telphone number"]
mail=["mail addresses","mail address","mail"]
location=["location adress","private postal address"]
health=["health data","medical records","health information","medical information","Medical Procedure names","Medical Procedure identification codes"]
personal=["personal data","personal","personal information","private data","private information"]
not_personal=["not personal data","not personal information","not personal","not private data","not private","not private information"]
passport=['passport','code']
Driving_license=['driving license','code']
social_security_number=['social security number']
Tax_file_number=['tax file number']
credit_card=['credit card number']
candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passport+Driving_license+social_security_number+Tax_file_number+credit_card


In [4]:
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]

In [5]:
# pipeline = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli")

# def predict(sentence : str):
#     text_translated=translate(sentence)
#     result_details=pipeline(text_translated, candidate_labels, multi_label=True)
#     result_details=dict(zip(result_details['labels'],result_details['scores']))
#     result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
#     pii_detected={name : result[name] for name in tresh(result)}
#     pii_detected=check(pii_detected,sentence)
#     if license_plate(sentence):
#             pii_detected["license_plate"]=0.9
#     return int(pii_detected!={}),np.mean(list(pii_detected.values()))

In [9]:
pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

def predict(sentence : str):
    text_translated=translate(sentence)
    result_details=pipeline(text_translated, candidate_labels, multi_label=True)
    result_details=dict(zip(result_details['labels'],result_details['scores']))
    result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
    pii_detected={name : result[name] for name in tresh(result)}
    pii_detected=check(pii_detected,sentence)
    if license_plate(sentence):
            pii_detected["license_plate"]=0.9
    if pii_detected=={}:
            mean=0
    else:
            mean=np.mean(list(pii_detected.values()))
    del text_translated,result_details,result
    return int(pii_detected!={}),mean



TypeError: object of type 'NoneType' has no len()

In [7]:
train_X, test_X, train_y, test_y = train_test_split(df['sentence'], df['label'], test_size=0.2, random_state=2)
del df

In [8]:
y_pred=[]
predicted_proba=[]
for sent in tqdm(test_X):
    result=predict(sent)
    y_pred.append(result[0])
    predicted_proba.append(result[1])

  9%|▉         | 5/56 [01:11<12:04, 14.20s/it]

dict_values([0.9108832120895386])


TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

In [ ]:
print(result)

(0, 0)


In [ ]:
conf = confusion_matrix(test_y, y_pred)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

In [ ]:
probas = clr.predict_proba(X_test)
fpr0, tpr0, thresholds0 = roc_curve(y_test, probas[:, 0], pos_label=clr.classes_[0], drop_intermediate=False)
fpr0.shape